In [1]:
from sklearn import neighbors
import numpy as np
import networkx as nx
import time
import sklearn.manifold
from scipy.spatial import distance
from sklearn.decomposition import KernelPCA
from sklearn.neighbors import NearestNeighbors
from sklearn.utils import check_array
from datetime import datetime
import random
from sklearn.utils.graph_shortest_path import graph_shortest_path
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import multiprocessing as mp
import os
from sklearn.preprocessing import MinMaxScaler

In [2]:
class KruskalStress:
    
    def __init__(self, full_matrix, reduced_matrix, k = 100, method = 'd'):
        
        self.k = k
        self.method = method
        self.full_matrix = full_matrix
        self.reduced_matrix = reduced_matrix
        self.value_of_stress = self.kruskal_stress()
        
    def create_distance_matrix(self, data_set):
        samples = data_set.shape[0]
        d = pdist(data_set)
        distances = {}
        for i in range(samples - 1):
            distances[i] = {i + n + 1: d for n, d in
                            enumerate(d[0:samples - i - 1])}
            d = d[samples - i - 1:]
        return distances
    
    def k_nearest(self, data_set):
        distance_matrix = self.create_distance_matrix(data_set)
        nodes = len(distance_matrix) + 1
        result = dict()
        for v in range(nodes):
            candidate_links = [distance_matrix[n][v] for n in range(0, v)] + [0] + [distance_matrix[v][n] for n in range(v + 1, nodes)]
            closest_neighbors = np.argsort(candidate_links)
            closest_neighbors = closest_neighbors[closest_neighbors != v][:self.k]
            result[v] = dict()
            for neighbor in closest_neighbors:
                _min, _max = min(v, neighbor), max(v, neighbor)
                result[_min][_max] =distance_matrix[_min][_max]
        return result
    
    def creating_shortest_path(self, data_set):
        k_nearest_matrix = self.k_nearest(data_set)
        g = nx.Graph()
        for v, edges in k_nearest_matrix.items():
            g.add_weighted_edges_from(((v, n, cost) for n, cost in edges.items()))
        if self.method == 'd':
            answer = dict(nx.all_pairs_dijkstra_path_length(g))
        else:
            answer = dict(nx.floyd_warshall(g))
        return answer
    
    def creating_dissimilarity_matrix(self, data_set):
        short_path_dict = self.creating_shortest_path(data_set)
        dissimilarities = np.zeros((data_set.shape[0], data_set.shape[0]))

        for node, edges in short_path_dict.items():
            for neighbor, dissimilarity in edges.items():
                dissimilarities[node, neighbor] = dissimilarity
        return dissimilarities
    
    def kruskal_stress(self):
        dissimilarity_matrix_x = self.creating_dissimilarity_matrix(self.full_matrix)
        dissimilarity_matrix_y = self.creating_dissimilarity_matrix(self.reduced_matrix)
        return np.sqrt(np.power(dissimilarity_matrix_x - dissimilarity_matrix_y, 2).sum() / np.power(dissimilarity_matrix_x, 2).sum())

In [3]:
class Particle():
    def __init__(self, n_dim, n_features_to_consider, data_matrix):
        self.n_dim = n_dim
        self.n_feat = n_features_to_consider
        self.data = data_matrix
        self.position_i = []
        self.velocity_i = []
        self.pos_best_i = []
        self.err_best_i = -1
        self.err_i = -1
    
        for i in range(0, self.n_dim):
            self.velocity_i.append(random.uniform(0,1))
            self.position_i.append(random.uniform(0,1))

    def evaluate(self,costFunc):
        indices = np.argsort(self.position_i)[-self.n_feat:]
        reduced_matrix = self.data.iloc[:,indices]
        self.err_i=costFunc(self.data, reduced_matrix).value_of_stress
        
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i
    
    def update_velocity(self,pos_best_g):
        w=0.5       # constant inertia weight
        c1=1        # cognative constant
        c2=2        # social constant

        for i in range(0,self.n_dim):
            r1=random.random()
            r2=random.random()

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social
    
    def update_position(self):
        for i in range(0,self.n_dim):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]

            # adjust maximum position if necessary
            if self.position_i[i]>1:
                self.position_i[i]=0.9

            # adjust minimum position if neseccary
            if self.position_i[i] < 0:
                self.position_i[i]=0.1

In [4]:
class PSO():
    def __init__(self, costFunc, n_dim, n_feat, data_set, num_particles, maxiter):
        
        err_best_g=-1                   # best error for group
        pos_best_g=[]                   # best position for group

        self.result_list = []
        
        swarm=[]
        for i in range(0,num_particles):
            swarm.append(Particle(n_dim, n_feat, data_set))

        # begin optimization loop
        i=0
        while i < maxiter:
            t1 = datetime.now()
            for j in range(0,num_particles):
                swarm[j].evaluate(costFunc)

                if swarm[j].err_i < err_best_g or err_best_g == -1:
                    pos_best_g=list(swarm[j].position_i)
                    err_best_g=float(swarm[j].err_i)
#                     print(err_best_g)
            t2 = datetime.now()
            delta_t = t2-t1
            self.result_list.append((n_feat, i, pos_best_g, err_best_g, delta_t))

            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position()
            i+=1

In [ ]:
n_iterations = #number-of-iterations
n_particles = #number-of-particles
number_of_features = #initial-number-of-features
n_features = #desired-number-of-features
repo = #matrix representing the repository

results = [pool.apply(PSO, args=row) for row in [(KruskalStress, number_of_features, n_features, 
                                                          repo, n_particles, n_iterations)]]